In [ ]:
#default_exp update

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

# Update
update the database

In [ ]:
#export
from s3bz.s3bz import S3
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict, hashDict, saveStringToFile, loadStringFromFile, saveDictToFile, loadDictFromFile
from nicHelper.exception import errorString
from dict_hash import dict_hash, sha256
from base64 import b64encode, b64decode
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses_jsonschema import JsonSchemaMixin
from dataclasses import dataclass
from jsonschema import validate
from typing import List
from datetime import datetime
from nicHelper.schema import validateUrl
import pandas as pd
import os, logging, json

In [ ]:
#hide
from villaProductDatabase.database import ProductDatabase
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
#export
import os
DBHASHLOCATION = '/tmp/database.hash'
DBCACHELOCATION = '/tmp/database.cache'
DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
VALUEUPDATESCHEMAURL = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/valueUpdate.json'
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
  print(DAX_ENDPOINT)
except KeyError as e:
  print(f'dax endpoint missing {e}')

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
class Updater:
  pass

In [ ]:
class Tester( Updater, ProductDatabase):
  class Meta:
    table_name = os.environ['DATABASE_TABLE_NAME']
    region = os.environ['REGION']
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass

## Update with dict

In [ ]:
#export
@add_class_method(Updater)
def updateWithDict(cls, originalObject:Updater, inputDict:dict ):
  data = originalObject.data
  data.update(inputDict)
  return cls.fromDict(data)

In [ ]:
# test
item = next(Tester.query(171670))
print(f'engname is {item.data.get("pr_engname")}')

updatedItem = Tester.updateWithDict(item,{'pr_engname':'testName'})
assert updatedItem.data.get('pr_engname') == 'testName'
print(updatedItem.data.get('pr_engname'))

engname is JIRAAT YOUNG KALE 200 G.
testName


## update with list of dict

In [ ]:
#hide
@dataclass_json(undefined=Undefined.INCLUDE)
@dataclass
class Product(JsonSchemaMixin):
  iprcode: int
  cprcode: int
#   data: CatchAll
@dataclass_json
@dataclass
class ValueUpdate(JsonSchemaMixin):
  items: List[Product]
print(json.dumps(Product.json_schema(), indent=2, sort_keys=True))
print(json.dumps(ValueUpdate.json_schema(), indent=2, sort_keys=True))

{
  "$schema": "http://json-schema.org/draft-06/schema#",
  "description": "Product(iprcode: int, cprcode: int)",
  "properties": {
    "cprcode": {
      "type": "integer"
    },
    "iprcode": {
      "type": "integer"
    }
  },
  "required": [
    "iprcode",
    "cprcode"
  ],
  "type": "object"
}
{
  "$schema": "http://json-schema.org/draft-06/schema#",
  "definitions": {
    "Product": {
      "description": "Product(iprcode: int, cprcode: int)",
      "properties": {
        "cprcode": {
          "type": "integer"
        },
        "iprcode": {
          "type": "integer"
        }
      },
      "required": [
        "iprcode",
        "cprcode"
      ],
      "type": "object"
    }
  },
  "description": "ValueUpdate(items: List[__main__.Product])",
  "properties": {
    "items": {
      "items": {
        "$ref": "#/definitions/Product"
      },
      "type": "array"
    }
  },
  "required": [
    "items"
  ],
  "type": "object"
}


In [ ]:
#export
class ValueUpdate2:
   #### divide work into chunks
  @staticmethod
  def chunks(l, n): return [l[x: x+n] for x in range(0, len(l), n)]
  ##### plot time elapsed for debugging
  @staticmethod
  def getTimeElasped(t0): return (t0-datetime.now()).total_seconds()

  t0 = datetime.now()

  ### validate input
  @staticmethod
  def validateInput(input_):
    return validateUrl(VALUEUPDATESCHEMAURL, 
                       input_)

In [ ]:
sampleProducts = {'items':[{ 'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]}
# t0 = datetime.now()
# ValueUpdate2.chunks(sampleProducts, 500)
# ValueUpdate2.getTimeElasped(t0)
# schema = requests.get(VALUEUPDATESCHEMAURL).json()
# validate(sampleProducts, schema)
ValueUpdate2.validateInput(sampleProducts).items

[{'cprcode': 171670,
  'iprcode': 171670,
  'oprcode': 171670,
  'ordertype': 'Y',
  'pr_abb': 'JIRAPAT YOUNG KALE 2',
  'pr_active': 'Y',
  'pr_cgcode': '05',
  'pr_code': '0171670',
  'pr_dpcode': '19',
  'pr_engname': 'JIRAAT YOUNG KALE 200 G.',
  'pr_ggcode': '057',
  'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_puqty': '1',
  'pr_sa_method': '1',
  'pr_sucode1': 'CM845',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1'}]

In [ ]:
#export
@add_static_method(ValueUpdate2)
def saveBatchToDynamodb(productBatch:list, dbClass:Updater, db:pd.DataFrame, itemsUpdated:dict):
  def checkIfInDb(db:pd.DataFrame, iprcode:int, cprcode:int)->dbClass:
    '''check if product is in the database, if not, create an empty class with the product code''' 
    if 'cprcode' in db.columns:
      incumbentDb = db[db['cprcode']==cprcode] ## filter for the cprcode
      if incumbentDb.empty:
        incumbentSeries = pd.Series() ## if empty, create an empty series
      else:
        incumbentSeries = incumbentDb.iloc[0] ## if not empty, pull the series
        
      #### create object from series or new object if series is empty ####
      if incumbentSeries.any:  incumbentItem = dbClass.fromSeries(incumbentSeries)
      else: incumbentItem = dbClass.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
        
    #### if cprcode doesnt exist, create empty object ####
    else: incumbentItem = dbClass.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
    return incumbentItem
      
  ###### main ######
  with dbClass.batch_write() as batch:
    # loop through each product
    for product in productBatch:
      iprcode = product['iprcode']
      cprcode = product['cprcode']
      incumbentItem = checkIfInDb(db, iprcode, cprcode) ### check if product in db or return empty obj

      ##### make a copy of original data
      originalData = incumbentItem.data.copy()
      ###### update data
      updatedData = dbClass.updateWithDict(incumbentItem, product)

      logging.info(f'incumbentBr is {incumbentItem.iprcode}\n, prcode is {iprcode}')

      # check for difference
      try:
        if updatedData.data != originalData:
          logging.info(f'product {iprcode} has changed from \n{originalData} \n{updatedData.data}')
          batch.save(updatedData)
          itemsUpdated['success'] += 1
        else:
          logging.info(f'no change for {iprcode}')
          itemsUpdated['skipped'] += 1
      except Exception as e:
        itemsUpdated['failure'] += 1
        itemsUpdated['failureMessage'].append(e)

    # log time taken
#     itemsUpdated['timetaken(ms)'] = (datetime.now()- t0).total_seconds()*1000

In [ ]:
#export
@add_class_method(Updater)
def valueUpdate2(cls, inputs:List[dict]):
    ''' 
      check for difference and batch update the changes in product data
    '''
    #### setup #####
    itemsUpdated = {'success':0, 'failure': 0, 'skipped': 0 ,'failureMessage':[], 'timetaken(ms)': 0}
    t0 = datetime.now()
    
    ##### main #####
    
    ######validate input ###########
    products = ValueUpdate2.validateInput(inputs).items
    print(f'products are {products}')
    print(f'time taken for validation  {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ######dividing input into batch of 500###########
    productBatches = ValueUpdate2.chunks(products, 500)
    print(f'divided into chunks {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ######try to load from cache########
    db = cls.loadFromCache().fillna('none')
    print(f'get all from s3 {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ###### save to dynamodb ########
    for productBatch in productBatches:
      ValueUpdate2.saveBatchToDynamodb(productBatch, cls, db, itemsUpdated)
    itemsUpdated['timetaken(ms)'] = (datetime.now()- t0).total_seconds()*1000
    return itemsUpdated

In [ ]:
%%time
# sampleProducts = {'items':[{ 'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]}
# products = ValueUpdate.from_dict({'items':sampleProducts}).to_dict()
Tester.valueUpdate2(sampleProducts)
# Tester.saveRemoteCache()

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
time taken for validation  -0.021606 ms
divided into chunks -0.022067 ms
loading hashkey allData-hash


<ipython-input-102-2bf83c032b62>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  incumbentSeries = pd.Series() ## if empty, create an empty series
INFO:root:incumbentBr is 
, prcode is 171670
INFO:root:product 171670 has changed from 
{} 
{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}


loaded hash is d0c0857f93ea21a99bc59a7c459dea6b8341d7cf
data is still in sync, using local cache
get all from s3 -0.262969 ms
CPU times: user 57.1 ms, sys: 4.28 ms, total: 61.3 ms
Wall time: 277 ms


{'success': 1,
 'failure': 0,
 'skipped': 0,
 'failureMessage': [],
 'timetaken(ms)': 277.072}

In [ ]:
db = Tester.loadFromCache()
db

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
loaded hash is d0c0857f93ea21a99bc59a7c459dea6b8341d7cf
data is still in sync, using local cache


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,pr_ggcode,pr_market,pr_name,pr_puqty,pr_sa_method,pr_sucode1,pr_suref3,prtype,psqty,pstype
0,0171670,0171670,0171670,Y,JIRAPAT YOUG KALE 2,Y,05,0171670,19,JIRAPAT YOUNG KALE 200 G.,057,JIRAPAT ยอดคะน้า 200 G.,JIRAPAT ยอดคะน้า 200 G.,1,1,CM845,A,I,1,1
